# Austin Deep Learning Meetup
### Tuesday Oct 18, 2016
____

# Data Munging for Deep Learning
### Danny Mulligan, danny@dannymulligan.com

INSERT LINK TO GITHUB LOCATION OF THIS PRESENTATION

# http://github.com/

More than half the time you spend working on machine learning is actually spent preparing the data.  And yet most machine learning demos focus only on the machine learning, and ignore the data prep.  "Here's some data I prepared earlier".

In this presentation we'll focus on the data prep steps in Python.

- Getting the data
- Saving and loading
- Grouping, filtering, sorting
- Feature generation
- Normalizing, fuzzing, binning and other data transforms
- Dealing with missing data
- Splitting data into training & test sets
- ...anything else you want to do to prepare your data

----

# What are the most important Python libraries for Data Munging?

## Python Standard Library
- Lots and lots of useful functions
- You need to have a good idea of what's in this library
- If you don't, spend an hour skimming the documentation
- Really, you should know what's in the standard library

### https://docs.python.org/3/library/

## Numpy
- High performance arrays
- Allows you to use highly optimized numerical libraries with almost zero effort

### https://docs.scipy.org/doc/numpy-1.11.0/reference/

## Pandas
- Numpy arrays with fancy indexing
- Lots of other data processing productivity tools

### http://pandas.pydata.org/pandas-docs/stable/

## Jupyter/IPython notebooks
- Great for data exploration, learning the libraries, experimenting with machine learning, etc
- **This** is a Jupyter notebook

### https://jupyter.readthedocs.io/en/latest/
____

# Warning:
### I wrote this notebook using Python 3.x but I never tested with Python 2.x.  If you have problems make sure you are using Python 3.x
### You MUST have Pandas & Numpy installed to run much of the code in this notebook.
#### I recommend installing Anaconda if you want these (plus a bunch of other very useful) libraries.
#### Download Anaconda from here: https://www.continuum.io/downloads

In [1]:
import datetime      # These imports are from the standard Python Library, they should always work
import json
import random
import sys
print("Python version", sys.version)

import numpy as np   # If this fails, you need to install Numpy
print("Numpy version", np.__version__)

import pandas as pd  # If this fails, you need to install Pandas
print("Pandas version", pd.__version__)

Python version 3.5.2 |Anaconda 4.0.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
Numpy version 1.11.0
Pandas version 0.18.0


# But wait, isn't Python too slow for processing large amounts of data?

#### It depends on what you're doing, but probably not.

#### If the heavy lifting happens inside a Python library, it will be just as fast as practically any other language.

In [2]:
# Make a list containing random numbers between 0.0 & 1.0
MyList = [random.random() for x in range(1000)]
MyList[:5]  # print the first 5 items in the list

[0.42666548276287775,
 0.1967159699222134,
 0.048362431018615926,
 0.934929106941457,
 0.8108076116906293]

In [3]:
def MyMax(list):
    '''Find the largest number in a list'''
    Max = list[0]
    for x in list[1:]:
        if x > Max:
            Max = x
    return Max

In [4]:
%timeit MyMax(MyList)
# When I ran this, I got...
# 10000 loops, best of 3: 40.1 µs per loop

10000 loops, best of 3: 36.7 µs per loop


In [5]:
%timeit max(MyList)
# When I ran this, I got...
# 10000 loops, best of 3: 25.9 µs per loop

10000 loops, best of 3: 24.6 µs per loop


In [6]:
MyList2 = np.array(MyList)
%timeit np.max(MyList2)

The slowest run took 43.74 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.13 µs per loop


### When I ran the above experiments, I got these results

#### 37.1 µs = MyMax function
#### 26.5 µs = Python library max (1.4x faster)
#### 5.55 µs = Numpy library max (6.7x faster)

### Try the same thing with sorting

In [7]:
# From: https://stackoverflow.com/questions/18262306/quick-sort-with-python
# Written by https://stackoverflow.com/users/3011380/zangw
def qsort(arr): 
     if len(arr) <= 1:
          return arr
     else:
          return qsort([x for x in arr[1:] if x<arr[0]]) + \
                 [arr[0]] +                                \
                 qsort([x for x in arr[1:] if x>=arr[0]])

In [8]:
%timeit MyList3 = qsort(MyList)
MyList3 = qsort(MyList)
MyList3[:5]

100 loops, best of 3: 3.52 ms per loop


[0.0009357069005411223,
 0.001999424943795458,
 0.0025996761140020253,
 0.005543592170138667,
 0.006831510143401109]

In [9]:
%timeit MyList4 = MyList.copy(); MyList4.sort()
MyList4 = MyList.copy(); MyList4.sort()
MyList4[:5]

1000 loops, best of 3: 216 µs per loop


[0.0009357069005411223,
 0.001999424943795458,
 0.0025996761140020253,
 0.005543592170138667,
 0.006831510143401109]

In [10]:
MyList5 = np.array(MyList)
%timeit MyList6 = np.sort(MyList5)
MyList6 = np.sort(MyList5)
MyList6[:5]

The slowest run took 5.34 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 28.2 µs per loop


array([ 0.00093571,  0.00199942,  0.00259968,  0.00554359,  0.00683151])

### When I ran the above sorting experiments, I got these results

#### 3,720 µs = qsort function
#### 227 µs = Python library sort (16.4x faster)
#### 28.9 µs = Numpy library sort (128.7x faster)

## Conclusion: Python is plenty fast enough, if you are using optimized libraries for the heavy lifting

____

# Getting data

- Before you can process data, you've got to get it, and then get it into Python.
- Lots and lots of data is available on the Internet, so lets get something from there.

Here are example of some things I found with a Google search for "interesting datasets csv"

http://statweb.stanford.edu/~sabatti/data.html

https://www.comptroller.texas.gov/transparency/open-data/search-datasets/

https://catalog.data.gov/dataset?res_format=CSV
    
### Since we're focusing on the data munging, not the data, we're going to work with something simple like stock quotes.

#### Stock quotes are available from Yahoo at:
    http://ichart.finance.yahoo.com/table.csv?d=6&e=1&f=2009&g=d&a=7&b=19&c=2004&ignore=.csv&s=[SymbolName]

With a little examination, we can figure out how to build a URL to fetch any data we want at this link.

In [11]:
def MakeYahooStockURL(Ticker, Start, End):
    '''Make a URL to download stock data from Yahoo'''
    HeadURL = 'http://ichart.finance.yahoo.com/table.csv?'
    StartURL = '&a={}&b={}&c={}'.format(Start.month-1, Start.day, Start.year)
    EndURL   = '&d={}&e={}&f={}'.format(  End.month-1,   End.day,   End.year)
    TailURL = '&g=d&ignore=.csv&s={}'.format(Ticker)
    return HeadURL + StartURL + EndURL + TailURL

DateA = datetime.date(2001, 10,  1)
DateB = datetime.date(2001, 12, 31)
DateC = datetime.date(2016,  7,  1)
DateD = datetime.date(2016, 10,  1)

AppleURL = MakeYahooStockURL('AAPL', DateA, DateB)
print(AppleURL)
GoogleURL = MakeYahooStockURL('GOOG', DateC, DateD)
print(GoogleURL)

http://ichart.finance.yahoo.com/table.csv?&a=9&b=1&c=2001&d=11&e=31&f=2001&g=d&ignore=.csv&s=AAPL
http://ichart.finance.yahoo.com/table.csv?&a=6&b=1&c=2016&d=9&e=1&f=2016&g=d&ignore=.csv&s=GOOG


## Now let's download some data from that URL.

### This data is formatted very nicely and Pandas is smart enough to read it directly.
### For more complex data, we might use urllib.request.urlopen() from the standard Python Library.

In [12]:
AppleURL = MakeYahooStockURL('AAPL', datetime.date(2016, 3, 1), datetime.date(2016, 3, 7))  # 7 days of data
AAPL = pd.read_csv(AppleURL)

In [13]:
AAPL

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-03-07,102.389999,102.830002,100.959999,101.870003,35828900,100.707970
1,2016-03-04,102.370003,103.750000,101.370003,103.010002,46055100,101.834965
2,2016-03-03,100.580002,101.709999,100.449997,101.500000,36955700,100.342188
3,2016-03-02,100.510002,100.889999,99.639999,100.750000,33169600,99.600743
4,2016-03-01,97.650002,100.769997,97.419998,100.529999,50407100,99.383251


### If we were doing this a lot, we might write code that downloads the data to a local file or database, and read local data first if it exists.

# Other possible ways you might get data

### - Read data from a CSV file with csv.reader() or pd.read_csv()
### - Read data from an Excel spreadsheet with pd.read_excel()
### - Read data from a more complex URL with urllib.request.urlopen()
### - Read data from a database with SQLAlchemy
### - Read data from a JSON file with json.load()

# We're going to use stock data for the rest of the presentation

### Lets grab 10 years worth of stock ticker data for 10 stocks

In [14]:
StartDate = datetime.date(2006, 10,  1)
EndDate   = datetime.date(2016,  9, 30)

# Hack for development - get 0.5 years worth of data not 10
StartDate = datetime.date(2013, 10,  1)
EndDate   = datetime.date(2014,  3, 31)

Stocks = [
#    Symbol,  Name
    ('AAPL',  'Apple'    ),
    ('AMZN',  'Amazon'   ),
    ('DELL',  'Dell'     ),
    ('FB'  ,  'Facebook' ),
    ('GOOG',  'Google'   ),
    ('LNKD',  'LinkedIn' ),
    ('MSFT',  'Microsoft'),
    ('NFLX',  'Netflix'  ),
    ('TWTR',  'Twitter'  ),
    ('VMW' ,  'VMWare'   ),
]

HaveInternet = True

if HaveInternet:
    # Read data from the Internet
    StockData = pd.DataFrame()

    for Stock in Stocks:
        (Ticker, Name) = Stock
        URL = MakeYahooStockURL(Ticker, StartDate, EndDate)
        Temp = pd.read_csv(URL, parse_dates=['Date'])
        Temp.set_index('Date', inplace=True)
        StockData[Name] = Temp['Adj Close']
    
    # Save data for later in case I don't have Internet access
    StockData.to_csv("Backup_in_case_I_dont_have_Internet.csv")
else:
    # Read data saved from earlier when I had Internet access
    StockData = pd.read_csv("Backup_in_case_I_dont_have_Internet.csv", index_col=['Date'], parse_dates=['Date'])

### DELL went private 2013-10-30 so it doesn't appear in the recent dates

In [15]:
StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
Date,,,,,,,,,,
2014-03-31,73.037003,336.369995,NaN,60.240002,556.972503,184.940002,38.343580,50.290001,46.669998,108.019997
2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,106.290001
2014-03-27,73.134980,338.470001,NaN,60.970001,558.462551,188.539993,36.818816,52.025715,46.320000,106.660004
2014-03-26,73.450674,343.410004,NaN,60.389999,565.420539,185.929993,37.221054,53.182858,44.430000,105.559998
2014-03-25,74.159626,354.709991,NaN,64.889999,578.782219,184.330002,37.735544,52.977142,47.880001,110.080002


### Several of these companies went public in the past 10 years so they don't appear in the earliest dates

In [16]:
StockData.tail()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
Date,,,,,,,,,,
2013-10-07,65.592817,310.029999,13.84,50.520000,432.438319,237.210007,30.685502,45.451427,NaN,80.699997
2013-10-04,64.958069,319.040009,13.84,51.040001,435.739997,244.990005,31.219966,46.751427,NaN,81.059998
2013-10-03,65.009170,314.760010,13.83,49.180000,437.608153,245.070007,31.201535,45.959999,NaN,79.849998
2013-10-02,65.836223,320.510010,13.83,50.279999,443.552202,251.190002,31.256822,47.247143,NaN,82.440002
2013-10-01,65.621057,320.950012,13.84,50.419998,443.057714,251.000000,30.943520,46.374287,NaN,82.040001


### Pandas is able to keep track of the indices of data, and align everything correctly

### Pandas marks missing data with NaNs

In [17]:
StockData.describe()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare
count,125.000000,125.000000,21.000000,125.000000,125.000000,125.000000,125.000000,125.000000,98.000000,125.000000
mean,71.836215,365.242002,13.839952,56.630480,544.625390,216.609840,34.268996,53.066126,53.970816,90.134000
std,2.971797,27.138285,0.008891,8.021025,49.133799,15.755121,1.661849,6.250702,8.083490,9.346298
min,64.677001,298.230011,13.830000,44.820000,426.409319,184.330002,30.418270,41.204285,39.060001,77.239998
25%,70.292901,350.309998,13.830000,49.400002,515.190700,206.789993,33.452956,47.842857,47.444999,81.300003
50%,72.179735,364.940002,13.840000,55.049999,555.365586,216.619995,34.427858,51.871429,54.870001,89.070000
75%,74.023555,387.600006,13.845000,63.549999,581.929085,224.029999,35.282053,58.110001,59.410001,96.250000
max,77.113536,407.049988,13.860000,72.029999,609.476541,251.190002,38.343580,64.997147,73.309998,110.669998


## I wonder much much data we are missing?

#### Pandas has functions to handle missing data/NaNs, but we need to use a Numpy function to count them.

#### Pandas is built upon Numpy arrays, so we can just use Numpy functions on Python DataFrames without any fuss.

In [18]:
# Count the NaNs
np.isnan(StockData).sum()

Apple          0
Amazon         0
Dell         104
Facebook       0
Google         0
LinkedIn       0
Microsoft      0
Netflix        0
Twitter       27
VMWare         0
dtype: int64

In [19]:
# No, how many NaNs in the entire dataset?
np.isnan(StockData).sum().sum()

131

In [20]:
# Now many data samples do we have for each ticker?
StockData.count()

Apple        125
Amazon       125
Dell          21
Facebook     125
Google       125
LinkedIn     125
Microsoft    125
Netflix      125
Twitter       98
VMWare       125
dtype: int64

In [21]:
# How many data samples do we have all together?
StockData.count().sum()

1119

# Feature generation

### Lets focus on generating features from the date

In [22]:
# First we need to move the date from the index back to be a column
print("Our index columns are: {}".format(StockData.index.names))
print("Our data columns are: {}".format(StockData.columns))

print()
print("Resetting index...")
StockData.reset_index(inplace=True)
print()

print("Our index columns are: {}".format(StockData.index.names))
print("Our data columns are: {}".format(StockData.columns))

Our index columns are: ['Date']
Our data columns are: Index(['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare'],
      dtype='object')

Resetting index...

Our index columns are: [None]
Our data columns are: Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare'],
      dtype='object')


### Generate some date features using Pandas functions

#### Write code as if operating on a single row
#### Pandas then applies that code to every element in the series

In [23]:
StockData['Date-Weekday']     = StockData['Date'].dt.weekday  # day of the week with Monday=0, Sunday=6
StockData['Date-DayOfYear']   = StockData['Date'].dt.dayofyear
StockData['Date-DaysInMonth'] = StockData['Date'].dt.days_in_month
StockData['Date-DayOfMonth']  = StockData['Date'].dt.day
StockData['Date-Month']       = StockData['Date'].dt.month
StockData['Date-Quarter']     = StockData['Date'].dt.quarter
StockData['Date-Year']        = StockData['Date'].dt.year
StockData['Date-1st']         = StockData['Date'].dt.day == 1
StockData['Date-MonthStart']  = StockData['Date'].dt.is_month_start
StockData['Date-MonthEnd']    = StockData['Date'].dt.is_month_end

In [24]:
# The [] on a Pandas DataFrame picks out which columns (i.e. Pandas Series) we are referencing
StockData[['Date', 'Date-1st', 'Date-MonthStart', 'Date-MonthEnd']].head()

# An alternate way to do the same thing.
#Cols = ['Date', 'Date-1st', 'Date-MonthStart', 'Date-MonthEnd']
#StockData[Cols].head()

,Date,Date-1st,Date-MonthStart,Date-MonthEnd
0,2014-03-31,False,False,True
1,2014-03-28,False,False,False
2,2014-03-27,False,False,False
3,2014-03-26,False,False,False
4,2014-03-25,False,False,False


### Generate 1-hot variables for Monday to Friday

1-hot encoding is where you create a feature for every possible category of an input, and set it to 0 or 1 based on the input.

An example makes it easier to understand.

|DoW||Date-Mon|Date-Tue|Date-Wed|Date-Thu|Date-Fri|Date-Sat|Date-Sun|
|:-:||:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|Mon|| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
|Tue|| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
|Wed|| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
|Thu|| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
|Fri|| 0 | 0 | 0 | 0 | 1 | 0 | 0 |
|Sat|| 0 | 0 | 0 | 0 | 0 | 1 | 0 |
|Sun|| 0 | 0 | 0 | 0 | 0 | 0 | 1 |

It's hard to feed "Mon", "Tue", etc into a neural network, but 0s and 1s are no problem.

In [25]:
DayNames = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
for DoW, DayName in enumerate(DayNames):
    StockData['Date-{}'.format(DayName)] = (StockData['Date'].dt.weekday == DoW).astype(int)

In [26]:
StockData.head(20)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Quarter,Date-Year,Date-1st,Date-MonthStart,Date-MonthEnd,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri
0,2014-03-31,73.037003,336.369995,NaN,60.240002,556.972503,184.940002,38.343580,50.290001,46.669998,...,1,2014,False,False,True,1,0,0,0,0
1,2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,...,1,2014,False,False,False,0,0,0,0,1
2,2014-03-27,73.134980,338.470001,NaN,60.970001,558.462551,188.539993,36.818816,52.025715,46.320000,...,1,2014,False,False,False,0,0,0,1,0
3,2014-03-26,73.450674,343.410004,NaN,60.389999,565.420539,185.929993,37.221054,53.182858,44.430000,...,1,2014,False,False,False,0,0,1,0,0
4,2014-03-25,74.159626,354.709991,NaN,64.889999,578.782219,184.330002,37.735544,52.977142,47.880001,...,1,2014,False,False,False,0,1,0,0,0
5,2014-03-24,73.370393,351.850006,NaN,64.099998,578.387583,188.139999,37.885214,54.128571,48.770000,...,1,2014,False,False,False,1,0,0,0,0
6,2014-03-21,72.510393,360.619995,NaN,67.239998,590.930063,196.720001,37.567165,57.998573,50.919998,...,1,2014,False,False,False,0,0,0,0,1
7,2014-03-20,71.942963,368.970001,NaN,66.970001,597.983016,204.419998,37.726191,60.610001,50.119999,...,1,2014,False,False,False,0,0,0,1,0
8,2014-03-19,72.291316,373.230011,NaN,68.239998,599.027014,201.949997,36.734626,60.012856,51.240002,...,1,2014,False,False,False,0,0,1,0,0
9,2014-03-18,72.310363,378.769989,NaN,69.190002,605.025994,197.850006,36.996548,60.035713,51.130001,...,1,2014,False,False,False,0,1,0,0,0


### Generate 1-hot version of Quarter

In [27]:
for Quarter in range(1,5):
    StockData['Date-Quarter{}'.format(Quarter)] = (StockData['Date'].dt.quarter == Quarter).astype(int)

In [28]:
StockData.columns

Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DaysInMonth', 'Date-DayOfMonth', 'Date-Month',
       'Date-Quarter', 'Date-Year', 'Date-1st', 'Date-MonthStart',
       'Date-MonthEnd', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu',
       'Date-Fri', 'Date-Quarter1', 'Date-Quarter2', 'Date-Quarter3',
       'Date-Quarter4'],
      dtype='object')

In [29]:
StockData.head(10)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-MonthEnd,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Quarter1,Date-Quarter2,Date-Quarter3,Date-Quarter4
0,2014-03-31,73.037003,336.369995,NaN,60.240002,556.972503,184.940002,38.343580,50.290001,46.669998,...,True,1,0,0,0,0,1,0,0,0
1,2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,...,False,0,0,0,0,1,1,0,0,0
2,2014-03-27,73.134980,338.470001,NaN,60.970001,558.462551,188.539993,36.818816,52.025715,46.320000,...,False,0,0,0,1,0,1,0,0,0
3,2014-03-26,73.450674,343.410004,NaN,60.389999,565.420539,185.929993,37.221054,53.182858,44.430000,...,False,0,0,1,0,0,1,0,0,0
4,2014-03-25,74.159626,354.709991,NaN,64.889999,578.782219,184.330002,37.735544,52.977142,47.880001,...,False,0,1,0,0,0,1,0,0,0
5,2014-03-24,73.370393,351.850006,NaN,64.099998,578.387583,188.139999,37.885214,54.128571,48.770000,...,False,1,0,0,0,0,1,0,0,0
6,2014-03-21,72.510393,360.619995,NaN,67.239998,590.930063,196.720001,37.567165,57.998573,50.919998,...,False,0,0,0,0,1,1,0,0,0
7,2014-03-20,71.942963,368.970001,NaN,66.970001,597.983016,204.419998,37.726191,60.610001,50.119999,...,False,0,0,0,1,0,1,0,0,0
8,2014-03-19,72.291316,373.230011,NaN,68.239998,599.027014,201.949997,36.734626,60.012856,51.240002,...,False,0,0,1,0,0,1,0,0,0
9,2014-03-18,72.310363,378.769989,NaN,69.190002,605.025994,197.850006,36.996548,60.035713,51.130001,...,False,0,1,0,0,0,1,0,0,0


### We can deal with columns programmatically

In [30]:
# If we iterate through the Pandas DataFrame, we get the individual columns
for Column in StockData:
    print(Column)

# If you want to iterate through the rows in the DataFrame, use pd.DataFrame.iterrows()

Date
Apple
Amazon
Dell
Facebook
Google
LinkedIn
Microsoft
Netflix
Twitter
VMWare
Date-Weekday
Date-DayOfYear
Date-DaysInMonth
Date-DayOfMonth
Date-Month
Date-Quarter
Date-Year
Date-1st
Date-MonthStart
Date-MonthEnd
Date-Mon
Date-Tue
Date-Wed
Date-Thu
Date-Fri
Date-Quarter1
Date-Quarter2
Date-Quarter3
Date-Quarter4


In [31]:
# Rename columns
Columns = StockData.columns.values
Columns[-4:] = ['Date-Q{}'.format(N) for N in range(1,5)]
StockData.columns = Columns
print(StockData.columns)

Index(['Date', 'Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DaysInMonth', 'Date-DayOfMonth', 'Date-Month',
       'Date-Quarter', 'Date-Year', 'Date-1st', 'Date-MonthStart',
       'Date-MonthEnd', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu',
       'Date-Fri', 'Date-Q1', 'Date-Q2', 'Date-Q3', 'Date-Q4'],
      dtype='object')


In [32]:
# Several different ways to delete columns
del(StockData['Date-1st'])
StockData = StockData.drop(['Date-DaysInMonth'], axis=1)
StockData.drop(['Date-MonthStart', 'Date-MonthEnd'], axis=1, inplace=True)

In [33]:
# Move the 'Date' field back into the index
StockData = StockData.set_index('Date')

StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Year,Date-Mon,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-31,73.037003,336.369995,NaN,60.240002,556.972503,184.940002,38.343580,50.290001,46.669998,108.019997,...,2014,1,0,0,0,0,1,0,0,0
2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,106.290001,...,2014,0,0,0,0,1,1,0,0,0
2014-03-27,73.134980,338.470001,NaN,60.970001,558.462551,188.539993,36.818816,52.025715,46.320000,106.660004,...,2014,0,0,0,1,0,1,0,0,0
2014-03-26,73.450674,343.410004,NaN,60.389999,565.420539,185.929993,37.221054,53.182858,44.430000,105.559998,...,2014,0,0,1,0,0,1,0,0,0
2014-03-25,74.159626,354.709991,NaN,64.889999,578.782219,184.330002,37.735544,52.977142,47.880001,110.080002,...,2014,0,1,0,0,0,1,0,0,0


### All of the new features I created above were derived from a single column, but we can refer to multiple columns when generating features

In [34]:
StockNames = [StockName for _, StockName in Stocks]
print(StockNames)

StockData['MeanPrice'] = StockData[StockNames].mean(axis=1)  # axis=1 require to generate means per row
# Note that mean() is smart enough to deal with missing data

StockData['AMZN/AAPL'] = StockData['Amazon'] / StockData['Apple']

StockData[StockNames + ['MeanPrice', 'AMZN/AAPL']].head()

['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn', 'Microsoft', 'Netflix', 'Twitter', 'VMWare']


,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,MeanPrice,AMZN/AAPL
Date,,,,,,,,,,,,
2014-03-31,73.037003,336.369995,NaN,60.240002,556.972503,184.940002,38.343580,50.290001,46.669998,108.019997,161.653676,4.605474
2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,106.290001,162.721241,4.630727
2014-03-27,73.134980,338.470001,NaN,60.970001,558.462551,188.539993,36.818816,52.025715,46.320000,106.660004,162.378007,4.628018
2014-03-26,73.450674,343.410004,NaN,60.389999,565.420539,185.929993,37.221054,53.182858,44.430000,105.559998,163.221680,4.675383
2014-03-25,74.159626,354.709991,NaN,64.889999,578.782219,184.330002,37.735544,52.977142,47.880001,110.080002,167.282725,4.783061


# Grouping and filtering

### Pandas is great at selecting and grouping data
#### - Use .loc() and .iloc() to select specific data
#### - Use .groupby() to split the data into groups

In [35]:
# Show us only the data where 'Date-Fri' is 1
StockData.loc[StockData['Date-Fri'] == 1].head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-28,73.053333,338.290009,NaN,60.009998,559.992565,190.589996,37.698126,51.267143,47.299999,106.290001,...,0,0,0,1,1,0,0,0,162.721241,4.630727
2014-03-21,72.510393,360.619995,NaN,67.239998,590.930063,196.720001,37.567165,57.998573,50.919998,109.209999,...,0,0,0,1,1,0,0,0,171.524021,4.973356
2014-03-14,71.397298,373.739990,NaN,67.720001,585.815220,196.779999,35.265990,60.641430,51.919998,105.500000,...,0,0,0,1,1,0,0,0,172.086658,5.234652
2014-03-07,72.179735,372.059998,NaN,69.800003,606.789247,206.789993,35.453078,64.052856,53.529999,101.510002,...,0,0,0,1,1,0,0,0,175.796101,5.154632
2014-02-28,71.608214,362.100006,NaN,68.459999,607.218830,204.039993,35.836607,63.661430,54.910000,96.050003,...,0,0,0,1,1,0,0,0,173.765009,5.056683


In [36]:
# Show us Twitter prices on Fridays in Q2
StockData.loc[(StockData['Date-Fri'] == 1) & (StockData['Date-Q2'] == 1), 'Twitter']
# Note we use the boolean '&' here, not the logical 'and' which will throw
# a complicated error message.

Series([], Name: Twitter, dtype: float64)

In [37]:
# Show us days when the Netflix price was more than 2.5x higher than the Microsoft price
StockA = 'Netflix'
StockB = 'Microsoft'
Multiplier = 1.5

DayCount = StockData.loc[(StockData[StockA] > Multiplier*StockData[StockB]), [StockA, StockB]].count()[0]
print("The {a} price was more than {m}x higher than the {b} price on {d} days".format(
        a=StockA, b=StockB, m=Multiplier, d=DayCount))

StockData.loc[(StockData[StockA] > Multiplier*StockData[StockB]), [StockA, StockB]]

The Netflix price was more than 1.5x higher than the Microsoft price on 61 days


,Netflix,Microsoft
Date,,
2014-03-21,57.998573,37.567165
2014-03-20,60.610001,37.726191
2014-03-19,60.012856,36.734626
2014-03-18,60.035713,36.996548
2014-03-17,60.388573,35.593392
2014-03-14,60.641430,35.265990
2014-03-13,61.437141,35.443722
2014-03-12,62.368572,35.799189
2014-03-11,62.497143,35.565330


### GroupBy splits the data into groups that we specify

In [38]:
StockData.columns

Index(['Apple', 'Amazon', 'Dell', 'Facebook', 'Google', 'LinkedIn',
       'Microsoft', 'Netflix', 'Twitter', 'VMWare', 'Date-Weekday',
       'Date-DayOfYear', 'Date-DayOfMonth', 'Date-Month', 'Date-Quarter',
       'Date-Year', 'Date-Mon', 'Date-Tue', 'Date-Wed', 'Date-Thu', 'Date-Fri',
       'Date-Q1', 'Date-Q2', 'Date-Q3', 'Date-Q4', 'MeanPrice', 'AMZN/AAPL'],
      dtype='object')

In [39]:
# GroupBy will split the data by whatever criteria you specify
for Quarter, Data in StockData.groupby('Date-Quarter'):
    print("Q{} mean prices".format(Quarter))
    print("There are {} rows in this group of data".format(len(Data)))
    print(Data[StockNames].mean())
    print()

Q1 mean prices
There are 61 rows in this group of data
Apple         72.304260
Amazon       371.143772
Dell                NaN
Facebook      63.368688
Google       583.956637
LinkedIn     205.040983
Microsoft     34.982583
Netflix       57.261335
Twitter       56.587049
VMWare        98.038525
dtype: float64

Q4 mean prices
There are 64 rows in this group of data
Apple         71.390109
Amazon       359.616877
Dell          13.839952
Facebook      50.208125
Google       507.137794
LinkedIn     227.636407
Microsoft     33.588859
Netflix       49.067567
Twitter       49.657568
VMWare        82.600000
dtype: float64



In [40]:
for (Year, Quarter), Data in StockData.groupby(['Date-Year','Date-Quarter']):
    print("{}-Q{} mean prices".format(Year, Quarter))
    print(Data[StockNames].mean())
    print()

2013-Q4 mean prices
Apple         71.390109
Amazon       359.616877
Dell          13.839952
Facebook      50.208125
Google       507.137794
LinkedIn     227.636407
Microsoft     33.588859
Netflix       49.067567
Twitter       49.657568
VMWare        82.600000
dtype: float64

2014-Q1 mean prices
Apple         72.304260
Amazon       371.143772
Dell                NaN
Facebook      63.368688
Google       583.956637
LinkedIn     205.040983
Microsoft     34.982583
Netflix       57.261335
Twitter       56.587049
VMWare        98.038525
dtype: float64



# Normalizing

### Neural networks often perform better if the inputs have a mean of 0.0, and standard-deviation of 1.0

In [41]:
# Create a data structure to record what normalization we did
DataDescription = {}

In [42]:
for StockName in StockNames:
    # Calculate the mean and std-dev of each stock
    Mean = StockData[StockName].mean()
    StdDev = StockData[StockName].std()

    # Record the results in our data description
    DataDescription["{}-mean"  .format(StockName)] = Mean
    DataDescription["{}-stddev".format(StockName)] = StdDev
    
    # Renormalize the stock data
    print("{} mean = {:.2f}, std-dev = {:.2f}".format(StockName, Mean, StdDev))
    StockData[StockName] = (StockData[StockName] - Mean) / StdDev

Apple mean = 71.84, std-dev = 2.97
Amazon mean = 365.24, std-dev = 27.14
Dell mean = 13.84, std-dev = 0.01
Facebook mean = 56.63, std-dev = 8.02
Google mean = 544.63, std-dev = 49.13
LinkedIn mean = 216.61, std-dev = 15.76
Microsoft mean = 34.27, std-dev = 1.66
Netflix mean = 53.07, std-dev = 6.25
Twitter mean = 53.97, std-dev = 8.08
VMWare mean = 90.13, std-dev = 9.35


In [43]:
DataDescription

{'Amazon-mean': 365.2420017359999,
 'Amazon-stddev': 27.138285329755124,
 'Apple-mean': 71.836215144,
 'Apple-stddev': 2.971797186622529,
 'Dell-mean': 13.839952380952381,
 'Dell-stddev': 0.008890872794479577,
 'Facebook-mean': 56.630479816,
 'Facebook-stddev': 8.021024560884324,
 'Google-mean': 544.625389712,
 'Google-stddev': 49.13379896280838,
 'LinkedIn-mean': 216.609839672,
 'LinkedIn-stddev': 15.755120924262556,
 'Microsoft-mean': 34.26899624800001,
 'Microsoft-stddev': 1.6618487866025151,
 'Netflix-mean': 53.066125775999986,
 'Netflix-stddev': 6.250702264548309,
 'Twitter-mean': 53.97081623469388,
 'Twitter-stddev': 8.083489881748925,
 'VMWare-mean': 90.13400019199992,
 'VMWare-stddev': 9.346298074193964}

### It's a good idea to save normalizing data in file
If you've trained a neural network model on normalized data, you MUST use the same normalization parameters when you're running the model

#### We'll save this data in a JSON file

In [44]:
# Save the data description in a file
DataDescriptionFileName = "DataDescription.json"
with open(DataDescriptionFileName, 'w') as f:
    json.dump(DataDescription, f, indent=4)
print("Saved data description to {}".format(DataDescriptionFileName))

Saved data description to DataDescription.json


In [45]:
!head DataDescription.json

{
    "Microsoft-mean": 34.26899624800001,
    "LinkedIn-stddev": 15.755120924262556,
    "Amazon-mean": 365.2420017359999,
    "Apple-stddev": 2.971797186622529,
    "Google-stddev": 49.13379896280838,
    "VMWare-mean": 90.13400019199992,
    "LinkedIn-mean": 216.609839672,
    "Twitter-mean": 53.97081623469388,
    "Twitter-stddev": 8.083489881748925,


# Other transformations

### This presentation is already too long, so I can't cover these, but other things you might do include...

#### - Stacking and unstacking data
Moving data from rows to columns or visa-versa
#### - Filling in missing data by various means
#### - Calculating rolling statistics
#### - Resampling data
Say from daily to weekly or monthly
#### - Sorting, dealing with duplicates, looking for unique values
#### - Lots more

# Partitioning data into train/test sets

### Normal practice is to present training data to a neural network in mini-batches
### It is also normal to hold back a portion of the data to use as test data

In [46]:
# Mark everything as 'train' data, then overwrite some of it as 'test' data later
StockData['Set'] = 'train'

### Also, let's assume we can't use the initial data in our training or test sets, because we don't have enough previous history to do something useful with it.

In [47]:
# We want to find the first date in the table 
# We do this by...
#    Grabbing the index which contains the date with "StockData.index"
#    Sorting it with "np.sort()"
#    Grabbing the first (i.e. earliest) date with "[0]"
#    Converting from Numpy timestamps to datetime.datetime with "pd.to_datetime()"
#    Converting from datetime.datetime to datetime.date with ".date()"
# Ugh, I'm not going to sugar coat this one, this code is UGLY
FirstDate = pd.to_datetime(np.sort(StockData.index)[0]).date()

DaysHistoryRequired = 90
StartDate = FirstDate + datetime.timedelta(days=DaysHistoryRequired)
print("Dates prior to {} will be marked as history".format(StartDate))

if StartDate == datetime.date(1970,4,1):
    print("Warning, you are getting a bad start date because you're running this code while 'Date' is not in the Index")

Dates prior to 2013-12-30 will be marked as history


In [48]:
# We need 'Date' as a column for a bit
StockData.reset_index(inplace=True)

In [49]:
StockData.loc[StockData.Date < StartDate, 'Set'] = 'history'

print("We now have {:,} rows marked as 'history'".format(len(StockData.loc[StockData.Set == 'history'])))
print("We now have {:,} rows marked as 'train'".format(len(StockData.loc[StockData.Set == 'train'])))

We now have 62 rows marked as 'history'
We now have 63 rows marked as 'train'


In [50]:
# Move 'Date' back to the index
StockData.set_index(['Date'], inplace=True)

In [51]:
StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-31,0.404061,-1.063885,NaN,0.450008,0.251296,-2.010130,2.451838,-0.444130,-0.903177,1.913699,...,0,0,0,1,0,0,0,161.653676,4.605474,train
2014-03-28,0.409556,-0.993135,NaN,0.421332,0.312762,-1.651517,2.063443,-0.287805,-0.825240,1.728599,...,0,0,1,1,0,0,0,162.721241,4.630727,train
2014-03-27,0.437030,-0.986503,NaN,0.541018,0.281622,-1.781633,1.534327,-0.166447,-0.946474,1.768187,...,0,1,0,1,0,0,0,162.378007,4.628018,train
2014-03-26,0.543260,-0.804472,NaN,0.468708,0.423235,-1.947294,1.776370,0.018675,-1.180284,1.650493,...,1,0,0,1,0,0,0,163.221680,4.675383,train
2014-03-25,0.781820,-0.388087,NaN,1.029734,0.695180,-2.048847,2.085959,-0.014236,-0.753488,2.134107,...,0,0,0,1,0,0,0,167.282725,4.783061,train


In [52]:
StockData.tail()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2013-10-07,-2.100883,-2.034469,0.005356,-0.761808,-2.283297,1.307522,-2.156330,-1.218215,NaN,-1.009384,...,0,0,0,0,0,0,1,140.718674,4.726585,history
2013-10-04,-2.314474,-1.702465,0.005356,-0.696978,-2.216100,1.801330,-1.834722,-1.010238,NaN,-0.970866,...,0,0,1,0,0,0,1,143.182164,4.911476,history
2013-10-03,-2.297278,-1.860176,-1.119393,-0.928869,-2.178078,1.806407,-1.845812,-1.136853,NaN,-1.100329,...,0,1,0,0,0,0,1,142.496541,4.841779,history
2013-10-02,-2.018978,-1.648298,-1.119393,-0.791729,-2.057101,2.194852,-1.812544,-0.930933,NaN,-0.823213,...,1,0,0,0,0,0,1,145.126934,4.868293,history
2013-10-01,-2.091380,-1.632085,0.005356,-0.774275,-2.067165,2.182793,-2.001070,-1.070574,NaN,-0.866011,...,0,0,0,0,0,0,1,144.916288,4.890961,history


### We're going to randomly mark 20% of the 'train' data as 'test'

In [53]:
TestRatio = 0.2

In [54]:
# Switch to an integer index by moving 'Date' back to being a column
StockData.reset_index(inplace=True)
StockData.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124])

In [55]:
SampleIndex = StockData.loc[StockData.Set == 'train'].index.values.copy()  # Make a copy to avoid modifying StockData.index
NumRows = len(SampleIndex)
NumTest = int(TestRatio * NumRows)
NumTrain = NumRows - NumTest
print("We have {:,} rows, we will use {:,} of them for training, {:,} for testing.".format(NumRows, NumTrain, NumTest))

We have 63 rows, we will use 51 of them for training, 12 for testing.


In [56]:
np.random.shuffle(SampleIndex)

In [57]:
TrainIndex = SampleIndex[:NumTrain]

In [58]:
TestIndex = SampleIndex[NumTrain:]
StockData.loc[TestIndex, 'Set'] = 'test'

In [59]:
StockData.head(10)

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
0,2014-03-31,0.404061,-1.063885,NaN,0.450008,0.251296,-2.010130,2.451838,-0.444130,-0.903177,...,0,0,0,1,0,0,0,161.653676,4.605474,train
1,2014-03-28,0.409556,-0.993135,NaN,0.421332,0.312762,-1.651517,2.063443,-0.287805,-0.825240,...,0,0,1,1,0,0,0,162.721241,4.630727,train
2,2014-03-27,0.437030,-0.986503,NaN,0.541018,0.281622,-1.781633,1.534327,-0.166447,-0.946474,...,0,1,0,1,0,0,0,162.378007,4.628018,train
3,2014-03-26,0.543260,-0.804472,NaN,0.468708,0.423235,-1.947294,1.776370,0.018675,-1.180284,...,1,0,0,1,0,0,0,163.221680,4.675383,train
4,2014-03-25,0.781820,-0.388087,NaN,1.029734,0.695180,-2.048847,2.085959,-0.014236,-0.753488,...,0,0,0,1,0,0,0,167.282725,4.783061,train
5,2014-03-24,0.516246,-0.493472,NaN,0.931242,0.687148,-1.807021,2.176021,0.169972,-0.643387,...,0,0,0,1,0,0,0,167.265752,4.795531,train
6,2014-03-21,0.226859,-0.170313,NaN,1.322714,0.942420,-1.262436,1.984638,0.789103,-0.377414,...,0,0,1,1,0,0,0,171.524021,4.973356,train
7,2014-03-20,0.035920,0.137370,NaN,1.289052,1.085966,-0.773707,2.080331,1.206884,-0.476381,...,0,1,0,1,0,0,0,174.379130,5.128646,train
8,2014-03-19,0.153140,0.294345,NaN,1.447386,1.107214,-0.930481,1.483667,1.111352,-0.337826,...,1,0,0,1,0,0,0,174.675091,5.162861,test
9,2014-03-18,0.159549,0.498483,NaN,1.565825,1.229309,-1.190713,1.641276,1.115009,-0.351434,...,0,0,0,1,0,0,0,175.725402,5.238115,train


### Create a function to supply training samples in batches

In [60]:
BatchSize = 10
TrainEpoch = 1
TrainBatchStart = 0
np.random.shuffle(TrainIndex)

def TrainingBatch():
    global TrainBatchStart
    global TrainEpoch
    
    if (TrainBatchStart + BatchSize) > NumTrain:
        # Done with this Epoch, start a new one
        TrainEpoch += 1
        np.random.shuffle(TrainIndex)
        TrainBatchStart = 0
        
    BatchIndex = TrainIndex[TrainBatchStart:TrainBatchStart + BatchSize]
    TrainBatchStart += BatchSize
    return StockData.loc[BatchIndex]

In [61]:
TrainingBatch()

,Date,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
31,2014-02-13,0.756178,-0.296334,NaN,1.333935,1.113822,-1.582967,0.391703,1.487538,0.309172,...,0,1,0,1,0,0,0,170.789929,4.821592,train
3,2014-03-26,0.543260,-0.804472,NaN,0.468708,0.423235,-1.947294,1.776370,0.018675,-1.180284,...,1,0,0,1,0,0,0,163.221680,4.675383,train
16,2014-03-07,0.115593,0.251232,NaN,1.641875,1.265195,-0.623280,0.712509,1.757679,-0.054533,...,0,0,1,1,0,0,0,175.796101,5.154632,train
37,2014-02-05,-0.841393,-0.692453,NaN,0.693118,0.537401,-0.142800,-0.608370,0.753221,1.484406,...,1,0,0,1,0,0,0,167.774222,4.996700,train
50,2014-01-16,1.054822,1.126010,NaN,0.069757,0.669764,0.885436,-0.010561,-0.909221,0.816378,...,0,1,0,1,0,0,0,175.287629,5.279379,train
15,2014-03-10,0.137570,0.194854,NaN,1.919894,1.232460,-0.900015,0.667476,1.565244,-0.011235,...,0,0,0,1,0,0,0,175.283790,5.128795,train
27,2014-02-20,0.148102,-0.569012,NaN,1.620680,1.156621,-1.473796,0.628074,1.450971,0.328965,...,0,1,0,1,0,0,0,170.716597,4.839758,train
20,2014-03-03,-0.007121,-0.201266,NaN,1.343909,1.142185,-0.962217,0.644960,1.694144,-0.032265,...,0,0,0,1,0,0,0,172.172986,5.009813,train
53,2014-01-13,0.211858,0.948402,NaN,-0.089824,0.331842,-0.194848,-1.077678,-0.791977,0.476178,...,0,0,0,1,0,0,0,169.472180,5.395372,train
6,2014-03-21,0.226859,-0.170313,NaN,1.322714,0.942420,-1.262436,1.984638,0.789103,-0.377414,...,0,0,1,1,0,0,0,171.524021,4.973356,train


In [62]:
TrainEpoch

1

# Saving and loading data

In [63]:
!rm StockData.csv StockData.csv.gz
StockDataFile = 'StockData.csv'

StockData.to_csv(StockDataFile, index=False)

In [64]:
!ls -l StockData*

-rw-r--r--  1 dannymulligan  staff  33592 Oct 13 17:08 StockData.csv


#### This file isn't big enough for it to matter, but sometimes we can save a lot of time & space by reading & writing compressed data

In [65]:
StockData.to_csv(StockDataFile + '.gz', compression='gzip', index=False)

In [66]:
!ls -l StockData*

-rw-r--r--  1 dannymulligan  staff  33592 Oct 13 17:08 StockData.csv
-rw-r--r--  1 dannymulligan  staff  14653 Oct 13 17:08 StockData.csv.gz


In [67]:
# Delete the datastructure and read it back in from a file
del(StockData)

In [68]:
%timeit StockData = pd.read_csv(StockDataFile, index_col=['Date'], parse_dates=['Date'])

100 loops, best of 3: 3.28 ms per loop


In [69]:
%timeit StockData = pd.read_csv(StockDataFile + '.gz', index_col=['Date'], parse_dates=['Date'])

100 loops, best of 3: 3.68 ms per loop


#### In this case, reading the compressed file takes 23.4 ms vs 14.9 ms for the uncompressed file, but for large sparse data, the compressed file will sometimes be much faster

In [70]:
StockData = pd.read_csv(StockDataFile + '.gz', index_col=['Date'], parse_dates=['Date'])
StockData.head(10)

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-31,0.404061,-1.063885,NaN,0.450008,0.251296,-2.010130,2.451838,-0.444130,-0.903177,1.913699,...,0,0,0,1,0,0,0,161.653676,4.605474,train
2014-03-28,0.409556,-0.993135,NaN,0.421332,0.312762,-1.651517,2.063443,-0.287805,-0.825240,1.728599,...,0,0,1,1,0,0,0,162.721241,4.630727,train
2014-03-27,0.437030,-0.986503,NaN,0.541018,0.281622,-1.781633,1.534327,-0.166447,-0.946474,1.768187,...,0,1,0,1,0,0,0,162.378007,4.628018,train
2014-03-26,0.543260,-0.804472,NaN,0.468708,0.423235,-1.947294,1.776370,0.018675,-1.180284,1.650493,...,1,0,0,1,0,0,0,163.221680,4.675383,train
2014-03-25,0.781820,-0.388087,NaN,1.029734,0.695180,-2.048847,2.085959,-0.014236,-0.753488,2.134107,...,0,0,0,1,0,0,0,167.282725,4.783061,train
2014-03-24,0.516246,-0.493472,NaN,0.931242,0.687148,-1.807021,2.176021,0.169972,-0.643387,1.992875,...,0,0,0,1,0,0,0,167.265752,4.795531,train
2014-03-21,0.226859,-0.170313,NaN,1.322714,0.942420,-1.262436,1.984638,0.789103,-0.377414,2.041022,...,0,0,1,1,0,0,0,171.524021,4.973356,train
2014-03-20,0.035920,0.137370,NaN,1.289052,1.085966,-0.773707,2.080331,1.206884,-0.476381,2.197233,...,0,1,0,1,0,0,0,174.379130,5.128646,train
2014-03-19,0.153140,0.294345,NaN,1.447386,1.107214,-0.930481,1.483667,1.111352,-0.337826,2.056001,...,1,0,0,1,0,0,0,174.675091,5.162861,test


In [71]:
StockData.describe()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Tue,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL
count,1.250000e+02,1.250000e+02,2.100000e+01,1.250000e+02,1.250000e+02,1.250000e+02,1.250000e+02,1.250000e+02,9.800000e+01,1.250000e+02,...,125.00000,125.000000,125.00000,125.00000,125.000000,125.0,125.0,125.000000,125.000000,125.000000
mean,2.259526e-15,3.559819e-15,-4.328548e-17,3.179679e-16,-1.989520e-16,1.278977e-16,-4.757084e-15,2.660983e-15,4.531523e-18,9.059420e-15,...,0.20800,0.192000,0.20000,0.20800,0.488000,0.0,0.0,0.512000,165.034658,5.081125
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.40751,0.395458,0.40161,0.40751,0.501867,0.0,0.0,0.501867,10.321824,0.263386
min,-2.409052e+00,-2.469279e+00,-1.119393e+00,-1.472440e+00,-2.406003e+00,-2.048847e+00,-2.317134e+00,-1.897681e+00,-1.844601e+00,-1.379584e+00,...,0.00000,0.000000,0.00000,0.00000,0.000000,0.0,0.0,0.000000,136.023100,4.557526
25%,-5.193201e-01,-5.502191e-01,-1.119393e+00,-9.014407e-01,-5.990721e-01,-6.232797e-01,-4.910436e-01,-8.356291e-01,-8.073019e-01,-9.451868e-01,...,0.00000,0.000000,0.00000,0.00000,0.000000,0.0,0.0,0.000000,159.141567,4.911476
50%,1.155933e-01,-1.112818e-02,5.355947e-03,-1.970423e-01,2.185908e-01,6.445732e-04,9.559339e-02,-1.911300e-01,1.112372e-01,-1.138419e-01,...,0.00000,0.000000,0.00000,0.00000,0.000000,0.0,0.0,1.000000,168.796021,5.106089
75%,7.360327e-01,8.238547e-01,5.677304e-01,8.626727e-01,7.592268e-01,4.709681e-01,6.095962e-01,8.069294e-01,6.728758e-01,6.543767e-01,...,0.00000,0.000000,0.00000,0.00000,1.000000,0.0,0.0,1.000000,172.172986,5.238115
max,1.775801e+00,1.540554e+00,2.254854e+00,1.919894e+00,1.319889e+00,2.194852e+00,2.451838e+00,1.908749e+00,2.392430e+00,2.197233e+00,...,1.00000,1.000000,1.00000,1.00000,1.000000,0.0,0.0,1.000000,176.698749,5.961421


In [72]:
StockData.head()

,Apple,Amazon,Dell,Facebook,Google,LinkedIn,Microsoft,Netflix,Twitter,VMWare,...,Date-Wed,Date-Thu,Date-Fri,Date-Q1,Date-Q2,Date-Q3,Date-Q4,MeanPrice,AMZN/AAPL,Set
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-31,0.404061,-1.063885,NaN,0.450008,0.251296,-2.010130,2.451838,-0.444130,-0.903177,1.913699,...,0,0,0,1,0,0,0,161.653676,4.605474,train
2014-03-28,0.409556,-0.993135,NaN,0.421332,0.312762,-1.651517,2.063443,-0.287805,-0.825240,1.728599,...,0,0,1,1,0,0,0,162.721241,4.630727,train
2014-03-27,0.437030,-0.986503,NaN,0.541018,0.281622,-1.781633,1.534327,-0.166447,-0.946474,1.768187,...,0,1,0,1,0,0,0,162.378007,4.628018,train
2014-03-26,0.543260,-0.804472,NaN,0.468708,0.423235,-1.947294,1.776370,0.018675,-1.180284,1.650493,...,1,0,0,1,0,0,0,163.221680,4.675383,train
2014-03-25,0.781820,-0.388087,NaN,1.029734,0.695180,-2.048847,2.085959,-0.014236,-0.753488,2.134107,...,0,0,0,1,0,0,0,167.282725,4.783061,train


# Workspace

https://www.reddit.com/r/MachineLearning/

https://www.reddit.com/r/MachineLearning/comments/54bpsb/yann_lecun_deep_learning_and_the_future_of_ai/

https://www.youtube.com/watch?v=wofXCQXq1pg

http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf